In [6]:
import re


def ints(text: str) -> list[int]:
    return [int(x) for x in re.findall("-?\\d+", text)]


def first(iterable):
    return next(iter(iterable))


def data(day: int, parser=str, sep="\n", example=False) -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    filename = f"2025/{day}-example.txt" if example else f"2025/{day}.txt"
    sections = open(filename).read().rstrip().split(sep)
    return [parser(section) for section in sections]

In [100]:
# Day 1: Secret Entrance
def rotate(rotations, dial=50):
    part1, part2 = 0, 0
    for rotation in rotations:
        dir, steps = rotation[0], int(rotation[1:])
        part2 += steps // 100  # Full rotations
        if dir == "R":
            new_dial = (dial + steps) % 100
            if new_dial < dial:
                part2 += 1
        else:
            new_dial = (dial - steps) % 100
            if new_dial > dial and dial != 0:
                part2 += 1
            if new_dial == 0:
                part2 += 1
        if new_dial == 0:
            part1 += 1
        dial = new_dial
    return part1, part2


part1, part2 = rotate(data(1))
print(f"Part 1: {part1}")  # 995
print(f"Part 2: {part2}")  # 54676

Part 1: 995
Part 2: 5847


In [ ]:
# Day 2: Gift Shop
def invalid_ids(ranges, part1=True):
    def invalid(id):
        id = str(id)
        middle = len(id) // 2
        first_half = id[:middle]
        second_half = id[middle:]
        return first_half == second_half

    def invalid_part2(id):
        # Substrings from pos 1 -> middle, check if they are repeated for the
        # rest of the string
        id = str(id)
        for i in range(1, len(id) // 2 + 1):
            if len(id) % i == 0:
                repetitions = len(id) // i - 1
                if id[:i] * repetitions == id[i:]:
                    return True  # repeated pattern
        return False

    def start_stop(_range):
        start, stop = _range.split("-")
        return range(int(start), int(stop) + 1)

    if part1:
        return sum(id for _range in ranges for id in start_stop(_range) if invalid(id))
    else:
        return sum(
            id for _range in ranges for id in start_stop(_range) if invalid_part2(id)
        )


print(f"Part 1: {invalid_ids(data(2, sep=','))}")  # 56660955519
print(f"Part 2: {invalid_ids(data(2, sep=','), part1=False)}")  # 79183223243

Part 1: 56660955519
Part 2: 79183223243


In [157]:
# Day 3

In [ ]:
# Day 4: Printing Department
def neighbors(point, grid):
    row, col = point
    potential = (
        (row - 1, col - 1),
        (row, col - 1),
        (row + 1, col - 1),
        (row - 1, col),
        (row + 1, col),
        (row - 1, col + 1),
        (row, col + 1),
        (row + 1, col + 1),
    )
    for r, c in potential:
        if 0 <= r < len(grid[0]) and 0 <= c < len(grid):
            yield (r, c)


def adjacent_paper_rolls(point, grid):
    return sum(1 for p in neighbors(point, grid) if grid[p[0]][p[1]] == "@")


def accessable_rolls(grid):
    result = 0
    for r, _ in enumerate(grid):
        for c, char in enumerate(grid[r]):
            p = (r, c)
            if char == "@" and adjacent_paper_rolls(p, grid) < 4:
                result += 1
    return result


def remove_roll(point, grid):
    r, c = point
    grid[r][c] = "."
    return grid


example = """..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.""".splitlines()
# grid = example
grid = data(4)

print(f"Part 1: {accessable_rolls(grid)}")  # 1346

Part 1: 1346
